# Darknet 19 Full precision

In [10]:
# select a GPU
import os
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

import numpy as np
import h5py
import matplotlib.pyplot as plt
import copy
from scipy.io import savemat,loadmat
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

import tensorflow
print('Tensorflow version = ',tensorflow.__version__)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler, History
from tensorflow.keras.optimizers.schedules import PolynomialDecay
#from tensorflow.keras import backend as K

from darknet import darknet19
from Utils import cutout,LR_WarmRestart,GetDataGen,plot_history

Tensorflow version =  2.4.0-dev20200823


In [11]:
# Dataset
WhichDataSet = 'CIFAR10'
#WhichDataSet = 'CIFAR100'
#load and prepare data
if WhichDataSet == 'CIFAR10':
    (x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.cifar10.load_data()
else:
    (x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.cifar100.load_data()
num_classes = np.unique(y_train).shape[0]
K_train = x_train.shape[0]
input_shape = x_train.shape[1:]
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')#/255.0
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)
# Learning parameters 
init_lr = 0.1
epochs = 1 #300
batch_size = 125
My_wd=5e-4
UseCutout=True
Loss = 'categorical_crossentropy'
steps_per_epoch = int(np.floor(K_train / batch_size))
lr_fn = PolynomialDecay(initial_learning_rate=0.1, 
                        end_learning_rate=0.0000001,
                        decay_steps=int(steps_per_epoch*epochs), 
                        power=4.0)
Optimizer = SGD(learning_rate=lr_fn,
                momentum=0.9, 
                nesterov=False)
Metrics = ['accuracy']
ModelsPath = 'TrainedModels/Tensorflow.keras/'

In [12]:
print(input_shape)
print(y_test.shape)

(32, 32, 3)
(10000, 10)


In [13]:
#fdefine a datagen or generating training samples with flip and pad/crop augmentation, and if set to True, with cutout augmentation
dataGenerator = GetDataGen(UseCutout)

#define and compile the model
model = darknet19(input_shape=input_shape)
model.compile(loss=Loss ,optimizer = Optimizer, metrics = Metrics)

#print  the model
model.summary()

#define callbacks
history = History()
callbacks = [history]

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 32)        864       
_________________________________________________________________
norm_1 (BatchNormalization)  (None, 32, 32, 32)        128       
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 16, 16, 64)        18432     
_________________________________________________________________
norm_2 (BatchNormalization)  (None, 16, 16, 64)        256 

In [14]:
history = model.fit_generator(dataGenerator.flow(x_train, y_train, batch_size=batch_size),
                              validation_data=(x_test, y_test),
                              epochs=epochs,
                              verbose=1,
                              callbacks=callbacks,
                              steps_per_epoch =steps_per_epoch)

400/400 [==============================] - 1987s 5s/step - loss: 2.2014 - accuracy: 0.1836 - val_loss: 2.0622 - val_accuracy: 0.2368


In [15]:
#get final performance
y_pred = model.predict(x_test)
print('Test accuracy (%):', 100*sum(np.argmax(y_pred,-1)==np.argmax(y_test,-1))/y_test.shape[0])

Test accuracy (%): 23.68


In [16]:
#plot loss and accuracy
plot_history(history,epochs)


#plot learning rate schedule
plt.figure(figsize=(16,4))
plt.plot(np.arange(0,len(lr_scheduler.lr_used))/steps_per_epoch,lr_scheduler.lr_used)
plt.xlabel('epoch number')
plt.ylabel('learning rate')
plt.show()

TypeError: plot_history() takes 1 positional argument but 2 were given

In [17]:
#save the weigts used for updating
model.save_weights(ModelsPath+'Final_weights_'+WhichDataSet+'_32bit_model_v2.h5')